<a href="https://colab.research.google.com/github/Jeanne0410/defia/blob/rfm-jeanne/temporalite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

séparer par type d'achat

les clients achètent de manière générale du pgc et des PRODUITS FRAIS :
favoriser ce type d'achat

supprimer textile et bazar des calculs car il n'y a pas trop de recurrence c'est souvent des produits acheter aléatoirement au moment du besoin

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

train_dataframes = []
for i in tqdm(range(1, 2)):
    train_dataframes.append(pd.read_csv(f'train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv('products_data.csv')

# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv('test_data.csv')

100%|██████████| 1/1 [00:25<00:00, 25.75s/it]
<ipython-input-21-47f0d5653ac2>:15: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('products_data.csv')


In [ ]:
columns_to_keep = ['date', 'transaction_id', 'customer_id', 'product_id']
train_data = train_data[columns_to_keep]

In [ ]:
import pandas as pd

# Assurer que la colonne 'date' est en format datetime
train_data['date'] = pd.to_datetime(train_data['date'])

# Extraire les informations temporelles
train_data['month'] = train_data['date'].dt.month        # Mois de l'année
#train_data['day_of_week'] = train_data['date'].dt.weekday    # jour de la semaine (0 = lundi, 6 = dimanche)
#l'information sur les jours de la semaine aurai pu etre interéssante mais pour les données test nous ne savons pas sur quel jour ce passe les achats

# Afficher un aperçu des données
print(train_data.head())

# Suppression de la colonne date si elle n'est plus nécessaire pour éviter la multicolinéarité
train_data = train_data.drop(columns=['date'])


        date       transaction_id   customer_id     product_id  month
0 2023-11-15  Transaction_1730125  Household_39   Product_5362     11
1 2022-07-20  Transaction_1560535  Household_39  Product_67174      7
2 2022-07-20  Transaction_1560535  Household_39  Product_82254      7
3 2023-11-15  Transaction_1730125  Household_39   Product_3895     11
4 2022-07-20  Transaction_1560535  Household_39  Product_34014      7


On selectionne les produits de Janvier et Fevrier pour essayer de capter une saisonnalité annuelle

In [ ]:
# Filtrer pour les mois de janvier et fevrier
train_data_filtered = train_data[train_data['month'].isin([1, 2])]

# Vérifier les données filtrées
print(train_data_filtered['month'].value_counts())


month
1    754535
2    669639
Name: count, dtype: int64


On garde les produits qui sont les plus fréquemment achetés

In [ ]:
# Identifier les produits les plus fréquents
top_products = train_data_filtered['product_id'].value_counts().nlargest(1000).index
print(top_products)
#train_data_filtered['product_id'] = train_data_filtered['product_id'].apply(lambda x: x if x in top_products else 'autre')
# Convertir top_products en un ensemble pour des recherches plus rapides
top_products_set = set(top_products)

# Filtrer les lignes où product_id est dans top_products
train_data_filtered = train_data_filtered[train_data_filtered['product_id'].isin(top_products_set)]

# Réinitialiser l'index pour garder un DataFrame propre
train_data_filtered = train_data_filtered.reset_index(drop=True)
print(train_data_filtered.shape)
print(train_data.shape)


Index(['Product_42748', 'Product_23971', 'Product_57942', 'Product_40951',
       'Product_2793', 'Product_9790', 'Product_73736', 'Product_51497',
       'Product_63301', 'Product_36019', 'Product_1370', 'Product_22371',
       'Product_32922', 'Product_64067', 'Product_47749', 'Product_36725',
       'Product_33592', 'Product_39751', 'Product_18947', 'Product_60177',
       'Product_35439', 'Product_37906', 'Product_80972', 'Product_72686',
       'Product_70197', 'Product_79974', 'Product_42910', 'Product_892',
       'Product_72095', 'Product_54653', 'Product_41366', 'Product_49539',
       'Product_61960', 'Product_69958', 'Product_8582', 'Product_77951',
       'Product_74083', 'Product_35597', 'Product_61170', 'Product_17821',
       'Product_2119', 'Product_7217', 'Product_6880', 'Product_31203',
       'Product_59110', 'Product_4700', 'Product_62701', 'Product_61035',
       'Product_17626', 'Product_76686', 'Product_30680', 'Product_20421',
       'Product_82444', 'Product_28

<ipython-input-7-57a1f561f237>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_filtered['product_id'] = train_data_filtered['product_id'].apply(lambda x: x if x in top_products else 'autre')


In [ ]:
'''# Trier les données par customer_id et date
train_data = train_data.sort_values(by=['customer_id', 'date'])

# Extraire les 10 premières transactions de chaque client
first_50_purchases = train_data.groupby('customer_id').head(50)

# Extraire des variables temporelles des dates
first_50_purchases['date'] = pd.to_datetime(first_50_purchases['date'])
first_50_purchases['month'] = first_50_purchases['date'].dt.month

# Vérifier les données
print(first_50_purchases.head())

# Optionnel : Suppression de la colonne date si non nécessaire
#first_10_purchases = first_10_purchases.drop(columns=['date'])
'''

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [ ]:
# Séparer X et y
X = train_data_filtered.drop(columns=["product_id"])  # Toutes les colonnes sauf product_id
y = train_data_filtered["product_id"]  # La cible (product_id)

'''# Diviser les données en batchs
batch_size = 10000
batches = [
    train_data_filtered.iloc[i:i + batch_size]
    for i in range(0, train_data_filtered.shape[0], batch_size)
]'''
print(X.head())
print(X.shape)
print(y.shape)

         transaction_id     customer_id  month
7   Transaction_1318448   Household_416      2
13  Transaction_2376486  Household_1264      1
14  Transaction_2735684  Household_1750      2
17  Transaction_1686434  Household_1956      2
18  Transaction_2032633  Household_1956      1
(1424174, 3)
(1424174,)


In [ ]:
# Encodage de la variable cible (product_id)
product_encoder = LabelEncoder()
y_encoded = product_encoder.fit_transform(y)

# Encodage des colonnes catégorielles dans X
categorical_columns = X.select_dtypes(include=["object"]).columns


In [ ]:
# Appliquer OneHotEncoder sur les colonnes catégorielles
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X[categorical_columns] = encoder.fit_transform(X[categorical_columns])


In [ ]:
numerical_columns = X.select_dtypes(exclude=["object"]).columns

# Ajouter les colonnes numériques aux données encodées
X_encoded = pd.concat([X[categorical_columns], X[numerical_columns]], axis=1)

# Assurez-vous que toutes les colonnes sont bien au format float32 pour économiser de la mémoire
X_encoded = X_encoded.astype(np.float32)

# Conversion en matrice NumPy
X_train = X_encoded.to_numpy()
y_train = to_categorical(y_encoded, num_classes=len(product_encoder.classes_))

# Vérifiez les dimensions
print(f"Dimensions de y_train : {y_train.shape}")
print(f"Dimensions de X_train : {X_train.shape}")


In [ ]:
# Encodage des données de test
X_test = test_data[['transaction_id', 'customer_id']]  # Select only the columns present in test_data and used for training

# Get categorical columns present in both train and test data
categorical_columns_test = list(set(categorical_columns) & set(X_test.columns))

# Create missing columns in X_test and fill with a placeholder value (e.g., 0)
# We fill with 0 because the numerical columns are related to the training data and do not exist in the test data.
# Instead of 'missing', we use 0 to represent the absence of these features in the test data.
for col in numerical_columns:
    if col not in X_test.columns:
        X_test[col] = 0

# Create missing columns in X_test and fill with a placeholder value (e.g., 'missing')
for col in categorical_columns:
    if col not in X_test.columns:
        X_test[col] = 'missing'  # Or any other suitable placeholder

# Now you can safely apply the transform
X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_columns]))

# Continue with the rest of your code...
x_test_final = pd.concat([X_test_encoded, X_test[numerical_columns].reset_index(drop=True)], axis=1)
X_test_final = x_test_final.to_numpy()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

num_features = X_train.shape[1]  # Nombre de colonnes dans X
num_products = y_train.shape[1]  # Nombre de classes (produits)

model = Sequential()
model.add(Dense(128, input_dim=num_features, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(num_products, activation="softmax"))  # Prédictions des produits

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 123s 3ms/step - accuracy: 0.8760 - loss: 11.2075
Epoch 2/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 118s 3ms/step - accuracy: 0.8859 - loss: 0.8679
Epoch 3/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 123s 3ms/step - accuracy: 0.8860 - loss: 0.8677
Epoch 4/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 127s 3ms/step - accuracy: 0.8856 - loss: 0.8700
Epoch 5/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 137s 3ms/step - accuracy: 0.8850 - loss: 0.8738
Epoch 6/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 146s 3ms/step - accuracy: 0.8857 - loss: 0.8695
Epoch 7/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 133s 3ms/step - accuracy: 0.8860 - loss: 0.8677
Epoch 8/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 122s 3ms/step - accuracy: 0.8856 - loss: 0.8706
Epoch 9/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 119s 3ms/step - accuracy: 0.8855 - loss: 0.8705
Epoch 10/10
44506/44506 ━━━━━━━━━━━━━━━━━━━━ 143s 3ms/step - accuracy: 0.8855 - loss: 0.8704


In [ ]:
def predict_product(model, top_n=10):
    """
    Génère les produits recommandés pour chaque instance dans X_train.

    Args:
        model: Le modèle entraîné.
        top_n: Le nombre de produits recommandés (par défaut : 10).

    Returns:
        Une liste de produits recommandés pour chaque exemple d'entraînement.
    """
    # Prédire les probabilités pour les données d'entraînement
    prediction = model.predict(X_train)

    # Obtenir les indices des produits les plus probables
    top_products = np.argsort(prediction, axis=1)[:, -top_n:][:, ::-1]

    # Décoder les produits pour retourner les identifiants originaux
    return [product_encoder.inverse_transform(top_products[i]) for i in range(len(top_products))]


In [ ]:
# Générer les recommandations
predicted_products = predict_product(model)

44506/44506 ━━━━━━━━━━━━━━━━━━━━ 92s 2ms/step


KeyboardInterrupt: 

In [ ]:
def hitrate_at_k(test_data, predicted_products, k=10):
    """
    Calcule le Hitrate@K.

    Args:
        test_data: Le jeu de données de test contenant les produits réellement achetés.
        predicted_products: Liste des produits recommandés par le modèle.
        k: Le nombre de recommandations à considérer (par défaut : 10).

    Returns:
        Le score Hitrate@K.
    """
    hits = 0

    for i, true_product in enumerate(test_data["product_id"]):
        if true_product in predicted_products[i][:k]:  # Vérifier si le vrai produit est dans les top K recommandations
            hits += 1

    return hits / len(test_data)


# Calcul du Hitrate@10
hitrate = hitrate_at_k(test_data, predicted_products, k=10)
print(f"Hitrate@10 : {hitrate:.2f}")
